#### Task: get the transformer to respond to "What is love?" and "Love is what?" Answer: "Awesome"

In [1]:
# imports

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

/Users/aryamantepal/anaconda3/envs/torch-lightning/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/aryamantepal/anaconda3/envs/torch-lightning/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/aryamantepal/anaconda3/envs/torch-lightning/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/aryamantepal/anaconda3/envs/torch-lightning/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/aryamantepal/anaconda3/envs/torch-lightning/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/aryamantepal/anaconda3/envs/torch-lightning/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/aryamantepal/anaconda3/envs/torch-lightning/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan o

In [2]:
# data

token_to_id = {
    'what': 0,
    'is': 1,
    'love': 2,
    'awesome': 3,
    '<EOS>': 4,
}

id_to_token = dict(map(reversed, token_to_id.items()))

inputs = torch.tensor([[token_to_id['what'],
                      token_to_id['is'],
                      token_to_id['love'],
                      token_to_id['<EOS>'],
                      token_to_id['awesome']],
                       
                       [token_to_id['love'],
                      token_to_id['is'],
                      token_to_id['what'],
                      token_to_id['<EOS>'],
                      token_to_id['awesome']]])

labels = torch.tensor([[token_to_id['is'],
                      token_to_id['love'],
                      token_to_id['<EOS>'],
                      token_to_id['awesome'],
                      token_to_id['<EOS>']],
                       
                      [token_to_id['is'],
                       token_to_id['what'],
                      token_to_id['<EOS>'],
                      token_to_id['awesome'],
                      token_to_id['<EOS>']]])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [1]:
class PositionEncoding:
    def __init__(self, d_model = 2, max_len = 6):
        super().__init__()
        
        pe = torch.zeros(max_len, d_model)
        
        position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze()
        embedding_index = torch.arange(start=0, end=d_model, step=2).float()
        
        div_term = 1/torch.tensor(10000.0)**(embedding_index / d_model)
        
        pe[:, 0::2] = torch.sin(position*div_term)
        pe[:, 1::2] = torch.cos(position*div_term)
        
        # places on gpu
        self.register_buffer('pe', pe)
        
    def forward(self, word_embeddings):
        return word_embeddings + self.pe[:word_embeddings.size(0), :]

In [ ]:
# query, key, value for each token
class Attention(nn.Module):
    def __init__(self, d_model=2):
        super().__init__()
        
        
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        
        self.row_dim = 0
        self.col_dim = 1
    
    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):
        """Calculate Masked Self-Attention for each token; encodings can come from diff sources, allowing Encoder-Decoder Attention"""
        
        q = self.W_q(encodings_for_q)
        k = self.W_k(encodings_for_k)
        v = self.W_v(encodings_for_v)
        
        sims = torch.matmul(q, k.tranpose(dim0=self.row_dim, dim1=self.col_dim))
        
        scaled_sims = sims / torch.tensor(k.size(self.col_dim) * 0.5)
        
        if mask is not None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
        
        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)
        
        attention_scores = torch.matmul(attention_percents, v)
        
        return attention_scores

In [ ]:
from typing import Any


class DecoderOnlyTransformer(L.LightningModule):
    def __init__(self, num_tokens=4, d_model=2, max_len=6):
        # num_tokens: number of tokens in the vocabulary
        # d_model: number of values used to represent each token
        # max_len: maximum length of the input + output
        super().__init__()
        
        self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model) 
        self.pe = PositionEncoding(d_model=d_model, max_len=max_len)
        self.attention = Attention(d_model=d_model)
        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)
        self.loss = nn.CrossEntropyLoss()
        
    def forward(self, token_ids):
        word_embeddings = self.we(token_ids)
        position_encoded = self.pe(word_embeddings)
        
        # mask prevents early tokens looking at late tokens when calculating attention
        mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0))))
        mask = mask == 0
        
        self_attention_values = self.attention(position_encoded, position_encoded, position_encoded, mask=mask)
        
        residual_connections = position_encoded + self_attention_values
        
        fc_layer_output = self.fc_layer(residual_connections)
        
        return fc_layer_output
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)
    
    def training_step(self, batch, batch_idx):
        input_tokens, label = batch
        output = self.forward(input_tokens)
        loss = self.loss(output, labels[0])
        return loss

In [ ]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)
model_input = torch.tensor([token_to_id['what'],
                      token_to_id['is'],
                      token_to_id['love'],
                      token_to_id['<EOS>'],
                      token_to_id['awesome']])
input_length = model_input.size(dim=0)
predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

max_length = 6
for i in range(input_length, max_length):
    if (predicted_id==token_to_id["<EOS>"]):
        break
    model_input = torch.cat((model_input, predicted_id))
    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids =torch.cat((predicted_ids, predicted_id))

print("Predicted Tokens\n")
for id in predicted_ids:
    print('\t', id_to_token[id.item()])